### 🔹 Step 1: Import Both DataFrames

In [19]:
# Imports
import pandas as pd
import numpy as np
import os

# Base path
base_path = r'C:\Users\moein\anaconda3\4 - Python Fundamentals\02 Data'

# Paths to subfolders
input_path = os.path.join(base_path, 'Original Data')
output_path = os.path.join(base_path, 'Prepared Data')

# Define which columns to import (omit 'eval_set')
vars_list = ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']

# Load the product and orders data
df_prods = pd.read_csv(os.path.join(input_path, 'products.csv'))
df_ords = pd.read_csv(os.path.join(input_path, 'orders.csv'), usecols=vars_list)

# Rename order_dow to orders_day_of_week
df_ords.rename(columns={'order_dow': 'orders_day_of_week'}, inplace=True)

### 🔹 Step 2: Run .describe() on df_ords

In [20]:
df_ords.describe(include='all')

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


- `orders_day_of_week` and `order_hour_of_day` values fall within expected ranges.
- `days_since_prior_order` contains missing values.
- No suspicious min/max values were observed.

### 🔹 Step 3: Check for Mixed-Type Columns

In [21]:
for col in df_ords.columns:
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
    if weird.any():
        print(col)

C:\Users\moein\AppData\Local\Temp\ipykernel_11456\3444883850.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
C:\Users\moein\AppData\Local\Temp\ipykernel_11456\3444883850.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
C:\Users\moein\AppData\Local\Temp\ipykernel_11456\3444883850.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
C:\Users\moein\AppData\Local\Temp\ipykernel_11456\3444883850.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis=1)
C:\Users\moein\AppData\Local\Temp\ip

No mixed-type columns were found in df_ords.

### 🔹 Step 4: Fix Mixed-Type Columns (if any found)

In [13]:
df_ords['column_name'] = df_ords['column_name'].astype('str')

KeyError: 'column_name'

This is not needed since no mixed-typed columns were found in df_ords.

### 🔹 Step 5: Check for Missing Values

In [22]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

The column `days_since_prior_order` has missing values. These occur on users’ first orders, where there is no previous order to compare.

### 🔹 Step 6: Handle Missing Values

In [23]:
df_ords['days_since_prior_order'].fillna(0, inplace=True)

C:\Users\moein\AppData\Local\Temp\ipykernel_11456\3696883554.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ords['days_since_prior_order'].fillna(0, inplace=True)


Filled missing values in `days_since_prior_order` with 0 to indicate that no previous order existed for the user (i.e., their first order).

### 🔹 Step 7: Check for Duplicates

In [24]:
df_ords[df_ords.duplicated()]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


No duplicate rows were found in df_ords.

### 🔹 Step 8: Remove Duplicates (if needed)

In [25]:
df_ords = df_ords.drop_duplicates()

Removed full duplicate rows using drop_duplicates(). It ensures the data is clean and avoids skewed analysis.

### 🔹 Step 9: Export the Cleaned Data

In [26]:
# Export cleaned files
df_prods.to_csv(os.path.join(output_path, 'products_checked.csv'), index=False)
df_ords.to_csv(os.path.join(output_path, 'orders_checked.csv'), index=False)

### 🔹Summary:
- No mixed-type columns found.
- Filled missing values in `days_since_prior_order` with 0.
- No duplicate rows found in `df_ords`.
- Cleaned data exported as `products_checked.csv` and `orders_checked.csv`.
